In [1]:
# Import all necessary libraries

import numpy as np
import pandas as pd
import os
import cv2

from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Flatten, Dense, Activation, Dropout
from keras.preprocessing import image

from matplotlib import pyplot as plt

from sklearn.metrics import classification_report, accuracy_score

Using TensorFlow backend.
C:\Users\LASYA\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\LASYA\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\LASYA\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\LASYA\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
# Load the labeled test dataset
NaturalBlurSet = pd.read_excel("C:/Users/LASYA/CERTH_ImageBlurDataset/EvaluationSet/NaturalBlurSet.xlsx")
DigitalBlurSet = pd.read_excel("C:/Users/LASYA/CERTH_ImageBlurDataset/EvaluationSet/DigitalBlurSet.xlsx")

In [3]:
# Rename attribute names for the DigitalBlurSet
DigitalBlurSet = DigitalBlurSet.rename(index=str, columns={"Unnamed: 1": "Blur Label"})
DigitalBlurSet = DigitalBlurSet.rename(index=str, columns={"MyDigital Blur": "Image Name"})

In [4]:
# Model Building
model = Sequential()

# Layer 1
model.add(Convolution2D(32, (5, 5), input_shape=(192, 192, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

# Layer 2
model.add(Convolution2D(32, (5, 5),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))    
model.add(Dropout(0.25))

# Layer 3
model.add(Convolution2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))    
model.add(Flatten())

# Layer 4
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

# Layer 5
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

# Layer 6
model.add(Dense(2, activation="softmax"))

In [5]:
# Load the weights derived during training
model.load_weights('model_weights.h5')

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 188, 188, 32)      2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 94, 94, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 90, 90, 32)        25632     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 45, 45, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 45, 45, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 41, 41, 64)        51264     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 20, 20, 64)       

In [7]:
# Initialize array variables
Natural_predicted_value = []
Natural_original_value = []
Digital_predicted_value = []
Digital_original_value = []

# Iterate through all images and make predictions. The predictions are saved as a list
for name,label in zip(NaturalBlurSet["Image Name"],NaturalBlurSet["Blur Label"]):
    img_path=os.path.join('C:/Users/LASYA/CERTH_ImageBlurDataset/EvaluationSet/NaturalBlurSet',name +'.jpg')
    img=cv2.imread(img_path)
    img=cv2.resize(img,(192,192)).astype('float32')
    Natural_predicted_value.append(model.predict_classes(img.reshape(1,192,192,3)))
    Natural_original_value.append(label)

for name,label in zip(DigitalBlurSet["Image Name"],DigitalBlurSet["Blur Label"]):
    img_path=os.path.join('C:/Users/LASYA/CERTH_ImageBlurDataset/EvaluationSet/DigitalBlurSet',name)
    img=cv2.imread(img_path)
    img=cv2.resize(img,(192,192)).astype('float32')
    Digital_predicted_value.append(model.predict_classes(img.reshape(1,192,192,3)))
    Digital_original_value.append(label)

In [8]:
Natural_prediction = np.stack(Natural_predicted_value)
Natural_original = np.stack(Natural_original_value)
Digital_prediction = np.stack(Digital_predicted_value)
Digital_original = np.stack(Digital_original_value)

In [9]:
# Print the test accuracy
print("Test Accuracy Score for NaturalBlurSet: ",accuracy_score(Natural_prediction,Natural_original)*100,"%")
print("Test Accuracy Score for DigitalBlurSet: ",accuracy_score(Digital_prediction,Digital_original)*100,"%")

Test Accuracy Score for NaturalBlurSet:  41.099999999999994 %
Test Accuracy Score for DigitalBlurSet:  93.75 %


In [10]:
# Print the classification report for Natural Blur Dataset
print(classification_report(Natural_prediction,Natural_original,target_names=['Clear',"Blurred"]))

              precision    recall  f1-score   support

       Clear       0.00      0.00      0.00         0
     Blurred       1.00      0.41      0.58      1000

    accuracy                           0.41      1000
   macro avg       0.50      0.21      0.29      1000
weighted avg       1.00      0.41      0.58      1000



C:\Users\LASYA\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
# Print the classification report for Digital Blur Dataset

print(classification_report(Digital_original_value,Digital_predicted_value,target_names=['Clear',"Blurred"]))

              precision    recall  f1-score   support

       Clear       0.00      0.00      0.00        30
     Blurred       0.94      1.00      0.97       450

    accuracy                           0.94       480
   macro avg       0.47      0.50      0.48       480
weighted avg       0.88      0.94      0.91       480



C:\Users\LASYA\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
